In [1]:
# Run below commands
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
# Test the spark 
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])

df.show(3, False)

/content/spark-3.0.1-bin-hadoop2.7/python/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [5]:
# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.0.1


In [67]:
import os
import findspark
import pandas as pd
import pyspark
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import StopWordsRemover 
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext
from imblearn.over_sampling import SMOTE
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF,StopWordsRemover,IDF,Tokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from sklearn.model_selection import train_test_split
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from sklearn.metrics import confusion_matrix
from pyspark.ml.classification import LinearSVC


# import modules for feature transformation
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF,StopWordsRemover,IDF,Tokenizer
from pyspark.ml.feature import NGram
from pyspark.ml.feature import RegexTokenizer
#Tokenize into words

from nltk.stem.snowball import SnowballStemmer

In [91]:
TRAIN_FILE = "/content/drive/MyDrive/CS651Final/jigsaw-toxic-comment-classification-challenge/train.csv"
TEST_DATA_FILE = "/content/drive/MyDrive/CS651Final/jigsaw-toxic-comment-classification-challenge/test.csv"
TEST_LABEL = "/content/drive/MyDrive/CS651Final/jigsaw-toxic-comment-classification-challenge/test_labels.csv"

In [121]:
from pyspark.sql.types import ArrayType, StringType, IntegerType, StructType, StructField

customSchema = StructType([
  StructField("id", StringType(), True),
  StructField("comment_text", StringType(), True),
  StructField("toxic", IntegerType(), True),
  StructField("severe_toxic", IntegerType(), True),
  StructField("obscene", IntegerType(), True),
  StructField("threat", IntegerType(), True),
  StructField("insult", IntegerType(), True),
  StructField("identity_hate", IntegerType(), True)]
)

In [132]:
# train = spark.read.csv(TRAIN_FILE, header='true', sep=',', multiLine=True, escape="\"")
train = spark.read.csv(TRAIN_FILE, header='true', multiLine=True, escape="\"", schema=customSchema)
test = spark.read.csv(TEST_DATA_FILE, header='true')

In [134]:
train.count()

159571

In [135]:
train.na.drop()

DataFrame[id: string, comment_text: string, toxic: int, severe_toxic: int, obscene: int, threat: int, insult: int, identity_hate: int]

In [136]:
train.count()

159571

In [137]:
train.printSchema

<bound method DataFrame.printSchema of DataFrame[id: string, comment_text: string, toxic: int, severe_toxic: int, obscene: int, threat: int, insult: int, identity_hate: int]>

In [138]:
from pyspark.sql.functions import *
train_clean = train.withColumn("comment_text", regexp_replace(col("comment_text"), "[\n\r\W]", " "))
train_clean = train_clean.withColumn("comment_text", regexp_replace(col("comment_text"), r'\d+', ""))

In [139]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

from pyspark.sql.types import ArrayType, StringType

# Stem text
stemmer_udf = udf(lambda line: stemming(line), StringType())
df_stemmed = train_clean.withColumn("comment_text", stemmer_udf("comment_text"))

In [140]:
df_stemmed.show()

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
|0000997932d777bf|explan whi the ed...|    0|           0|      0|     0|     0|            0|
|000103f0d9cfb60f|d aww he match th...|    0|           0|      0|     0|     0|            0|
|000113f07ec002fd|hey man i m reall...|    0|           0|      0|     0|     0|            0|
|0001b41b1c6bb37e|more i can t make...|    0|           0|      0|     0|     0|            0|
|0001d958c54c6e35|you sir are my he...|    0|           0|      0|     0|     0|            0|
|00025465d4725e87|congratul from me...|    0|           0|      0|     0|     0|            0|
|0002bcb3da6cb337|cocksuck befor yo...|    1|           1|      1|     0|     1|            0|
|00031b1e95af7921|your vandal to th...|    0|     

In [141]:
def check_clean(toxic, severe_toxic, obscene, threat, insult, identity_hate):
    if (toxic + severe_toxic + obscene + threat + insult + identity_hate) > 0:
        return 0
    else:
        return 1

In [142]:
from pyspark.sql.types import IntegerType

In [143]:
mergeCols = udf(lambda toxic, severe_toxic, obscene, threat, insult, identity_hate: check_clean(toxic, severe_toxic, obscene, threat, insult, identity_hate), IntegerType())

In [144]:
df_clean = df_stemmed.withColumn("clean", mergeCols(df_stemmed["toxic"], df_stemmed["severe_toxic"], df_stemmed["obscene"], df_stemmed["threat"], df_stemmed["insult"], df_stemmed["identity_hate"]))

In [145]:
df_clean.show()

+----------------+--------------------+-----+------------+-------+------+------+-------------+-----+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|clean|
+----------------+--------------------+-----+------------+-------+------+------+-------------+-----+
|0000997932d777bf|explan whi the ed...|    0|           0|      0|     0|     0|            0|    1|
|000103f0d9cfb60f|d aww he match th...|    0|           0|      0|     0|     0|            0|    1|
|000113f07ec002fd|hey man i m reall...|    0|           0|      0|     0|     0|            0|    1|
|0001b41b1c6bb37e|more i can t make...|    0|           0|      0|     0|     0|            0|    1|
|0001d958c54c6e35|you sir are my he...|    0|           0|      0|     0|     0|            0|    1|
|00025465d4725e87|congratul from me...|    0|           0|      0|     0|     0|            0|    1|
|0002bcb3da6cb337|cocksuck befor yo...|    1|           1|      1|     0|     1|           

In [173]:
tokenizer = Tokenizer().setInputCol("comment_text").setOutputCol("words")
#Remove stopwords
remover= StopWordsRemover().setInputCol("words").setOutputCol("filtered").setCaseSensitive(False)
# ngram = NGram().setN(2).setInputCol("filtered").setOutputCol("ngrams")
#For each sentence (bag of words),use HashingTF to hash the sentence into a feature vector. 
hashingTF = HashingTF().setNumFeatures(1000).setInputCol("filtered").setOutputCol("rawFeatures")
#Create TF_IDF features
idf = IDF().setInputCol("rawFeatures").setOutputCol("features").setMinDocFreq(0)
# Create a Logistic regression model
lr = LinearSVC(labelCol="label", featuresCol="features", maxIter=20)
# Streamline all above steps into a pipeline
pipeline=Pipeline(stages=[tokenizer,remover,hashingTF,idf, lr])

In [149]:
df_clean = df_clean.drop('toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate')

In [180]:
df_clean = df_clean.withColumnRenamed("clean", "label")

In [181]:
df_clean.show()

+----------------+--------------------+-----+
|              id|        comment_text|label|
+----------------+--------------------+-----+
|0000997932d777bf|explan whi the ed...|    1|
|000103f0d9cfb60f|d aww he match th...|    1|
|000113f07ec002fd|hey man i m reall...|    1|
|0001b41b1c6bb37e|more i can t make...|    1|
|0001d958c54c6e35|you sir are my he...|    1|
|00025465d4725e87|congratul from me...|    1|
|0002bcb3da6cb337|cocksuck befor yo...|    0|
|00031b1e95af7921|your vandal to th...|    1|
|00037261f536c51d|sorri if the word...|    1|
|00040093b2687caa|align on this sub...|    1|
|0005300084f90edc|fair use rational...|    1|
|00054a5e18b50dd4|bbq be a man and ...|    1|
|0005c987bdfc9d4b|hey what is it ta...|    0|
|0006f16e4e9f292e|befor you start t...|    1|
|00070ef96486d6f9|oh and the girl a...|    1|
|00078f8ce7eb276d|juelz santana age...|    1|
|0007e25b2121310b|bye don t look co...|    0|
|000897889268bc93|redirect talk voy...|    1|
|0009801bd85e5806|the mitsurugi po

In [182]:
training_spark_df_binary, testing_spark_df_binary = df_clean.randomSplit([0.8, 0.2], seed = 2018)

In [183]:
paramGrid = ParamGridBuilder()\
    .addGrid(hashingTF.numFeatures,[1000]) \
    .addGrid(lr.regParam, [0.1]) \
    .build()

In [184]:
crossval = TrainValidationSplit(estimator=pipeline,
                            estimatorParamMaps=paramGrid,
                            evaluator=BinaryClassificationEvaluator().setMetricName('areaUnderPR'), # set area Under precision-recall curve as the evaluation metric
                            # 80% of the data will be used for training, 20% for validation.
                            trainRatio=0.8)

In [185]:
cvModel = crossval.fit(training_spark_df_binary)

In [186]:
train_prediction = cvModel.transform(training_spark_df_binary)
test_prediction = cvModel.transform(testing_spark_df_binary)

In [ ]:
pd_prediction = test_prediction.select("*").toPandas()
actual = pd_prediction["label"].tolist()
pred = pd_prediction["prediction"].tolist()

# pd_prediction_other_dataset = otherDatasetTest.select("*").toPandas()
# actual_otherdataset = pd_prediction_other_dataset["label"].tolist()
# pred_otherdataset = pd_prediction_other_dataset["prediction"].tolist()

tn, fp, fn, tp = confusion_matrix(actual, pred).ravel()
print(confusion_matrix(actual, pred))
print("true positive rate",tp / (tp + fp))
print("true negative rate",tn / (tn + fp))

# compute the accuracy score on training data
correct_train = train_prediction.filter(train_prediction.label == train_prediction.prediction).count()  
accuracy_train = correct_train/train_prediction.count() # store the training accuracy score
print('Training set accuracy {:.2%} data items: {}, correct: {}'.format(accuracy_train,train_prediction.count(), correct_train))
    
# Caculate the accuracy score for the best model 
correct_test = test_prediction.filter(test_prediction.label == test_prediction.prediction).count()  
accuracy_test = correct_test/test_prediction.count()
print('Testing set accuracy {:.2%} data items: {}, correct: {}'.format(accuracy_test, test_prediction.count(), correct_test))
    
# Caculate the accuracy score for the best model 
# correct_test_otherDataset = otherDatasetTest.filter(otherDatasetTest.label == otherDatasetTest.prediction).count()  
# accuracy_test_otherDataset = correct_test_otherDataset/otherDatasetTest.count()
# print('Testing set accuracy for other dataset is  {:.2%} data items: {}, correct: {}'.format(accuracy_test_otherDataset, otherDatasetTest.count(), correct_test_otherDataset))  

[[   50  3170]
 [    4 28675]]
true positive rate 0.900455330507144
true negative rate 0.015527950310559006
Training set accuracy 90.00% data items: 127672, correct: 114908
